Option 2: Vary negative ratios in train/val, use scale_pos_weight, fixed global test set

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Option 2 (global fixed test set, per-sweep scale_pos_weight, rounded metrics):
- Read full cems_with_fraction_balanced.parquet
- Build features, coerce to numeric, drop bad rows
- Create a single global test set (10%) with true 0/1 distribution (no resampling)
- Use the remaining 90% as a train/val pool
- Within the train/val pool:
    - Fix a positive sample set once
    - Sweep negative ratios (10%..100%, where 100% = 2x positives)
    - For each sweep:
        - Build a balanced train/val dataset with that negative count
        - Split into train/val; compute scale_pos_weight from TRAIN subset
        - Train LightGBM (sklearn API)
        - Choose threshold on val (max precision with recall ≥ floor)
        - Evaluate on the SAME global test set
- Save:
    - Balanced train/val parquet per sweep
    - IoU learning curve PNG per sweep
    - Feature importance PNG per sweep
    - Summary CSV with **rounded** test metrics across sweeps
"""

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve,
    jaccard_score, precision_score, recall_score, f1_score
)

# =====================================================
# CONFIG
# =====================================================
PARQUET_IN    = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction.parquet"
RANDOM_STATE  = 42

# Overall split: global test set (10%), rest train/val pool (90%)
TEST_SIZE_GLOBAL = 0.10
VAL_SIZE_OVERALL = 0.20   # overall val fraction of full dataset; derive inner val from this

THRESH_INIT   = 0.50      # for IoU logging during training
RECALL_FLOOR  = 0.80
TOP_N_IMPORT  = 30

# Negative ratio sweep within the train/val pool:
# 100% = 2x positives, 10% = 0.2x positives, etc.
PCT_STEPS = list(range(10, 101, 10))

# Optional: cap positives in train/val pool for speed (None = use all)
MAX_SAMPLES_POS_TRAINVAL = None

# LightGBM base params
LGB_PARAMS = dict(
    objective="binary",
    boosting_type="gbdt",
    learning_rate=0.05,
    num_leaves=48,
    min_data_in_leaf=100,
    feature_fraction=0.75,
    bagging_fraction=0.75,
    bagging_freq=5,
    lambda_l2=2.0,
    n_jobs=-1,
)

# Output root and option-specific dir
OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest"
OUT_DIR  = os.path.join(OUT_ROOT, "option2_scale_pos_weight")
os.makedirs(OUT_DIR, exist_ok=True)

# =====================================================
# LOAD & EARLY CLEAN (ONCE)
# =====================================================
print(f"Loading: {PARQUET_IN}")
df = pd.read_parquet(PARQUET_IN)

if "fraction" not in df.columns:
    raise ValueError("Expected column 'fraction' in dataset.")

df["fraction"] = df["fraction"].astype("float32").clip(0.0, 1.0)
before = len(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any").copy()
print(f"Dropped {before - len(df):,} rows with NaNs/±inf; {len(df):,} remain.")

# =====================================================
# FRACTION -> BINARY
# =====================================================
df["burned"] = (df["fraction"] > THRESH_INIT).astype(np.uint8)

print("\nClass counts before any splitting:")
print(df["burned"].value_counts(dropna=False))

# =====================================================
# FEATURE MATRIX (GLOBAL) + TYPE COERCION
# =====================================================
drop_cols = {"fraction", "burned", "bin", "year", "month", "latitude", "longitude"}
predictors = [c for c in df.columns if c not in drop_cols]

X_full = df[predictors].copy()
y_full = df["burned"].astype(np.uint8)

# Treat 'b1' as category if present
if "b1" in X_full.columns and not pd.api.types.is_categorical_dtype(X_full["b1"]):
    X_full["b1"] = X_full["b1"].astype("category")
    print("\nTreating 'b1' as pandas 'category'.")

# Coerce non-category columns to numeric and drop rows with NaNs
coerced = 0
for c in X_full.columns:
    if c == "b1" and pd.api.types.is_categorical_dtype(X_full[c]):
        continue
    if not np.issubdtype(X_full[c].dtype, np.number):
        X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
        coerced += 1

if coerced:
    pre = len(X_full)
    num_cols = [c for c in X_full.columns if not (c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]))]
    mask = X_full[num_cols].notna().all(axis=1)
    if "b1" in X_full.columns and pd.api.types.is_categorical_dtype(X_full["b1"]):
        mask &= X_full["b1"].notna()
    X_full = X_full.loc[mask].copy()
    y_full = y_full.loc[X_full.index]
    print(f"Coerced {coerced} column(s); dropped {pre - len(X_full):,} rows post-coercion.")

print(f"\nPredictor columns: {len(X_full.columns)}")

# Combine into a single DataFrame for convenient splitting
data = X_full.copy()
data["burned"] = y_full

# =====================================================
# GLOBAL TRAINVAL / TEST SPLIT (FIXED TEST SET)
# =====================================================
idx_trainval, idx_test = train_test_split(
    data.index,
    test_size=TEST_SIZE_GLOBAL,
    random_state=RANDOM_STATE,
    stratify=data["burned"],
)

trainval = data.loc[idx_trainval].copy()
test     = data.loc[idx_test].copy()

print("\nGlobal split sizes (true distribution in test):")
print(f"  Train/Val pool: {len(trainval):,}")
print(f"  Test (fixed)  : {len(test):,}")
print("\nTest set class counts (true distribution):")
print(test["burned"].value_counts())

# Extract global test X, y
X_test = test[predictors].copy()
y_test = test["burned"].astype(np.uint8)

# =====================================================
# TRAIN/VAL POOL: POS/NEG SPLITTING BASE
# =====================================================
pos_tv = trainval[trainval["burned"] == 1]
neg_tv = trainval[trainval["burned"] == 0]

n_pos_tv = len(pos_tv)
n_neg_tv = len(neg_tv)

print("\nTrain/Val pool class counts:")
print(trainval["burned"].value_counts())

if n_pos_tv == 0 or n_neg_tv == 0:
    raise ValueError("Train/Val pool has only one class; cannot proceed.")

# Fix positive sample within train/val pool (optionally capped)
target_pos = n_pos_tv
if MAX_SAMPLES_POS_TRAINVAL is not None:
    target_pos = min(target_pos, MAX_SAMPLES_POS_TRAINVAL)

pos_tv_s = pos_tv.sample(n=min(n_pos_tv, target_pos), random_state=RANDOM_STATE)
n_pos_eff = len(pos_tv_s)
print(f"\nEffective positive samples in train/val sweeps: {n_pos_eff:,}")

# =====================================================
# Custom IoU metric for logging (sklearn API)
# =====================================================
def lgb_iou_metric_skl(y_true, y_pred):
    y_hat = (y_pred >= THRESH_INIT).astype(np.uint8)
    iou = jaccard_score(y_true, y_hat, average="binary", zero_division=0)
    return ("IoU", iou, True)

summary_rows = []

# =====================================================
# SWEEP OVER NEGATIVE RATIOS IN TRAIN/VAL POOL
# =====================================================
for pct in PCT_STEPS:
    print("\n" + "=" * 70)
    print(f"== Option 2: NEGATIVE PERCENT = {pct}% (100% = 2x positives in TRAIN/VAL) ==")

    # 100% -> 2 * n_pos_eff
    # 10%  -> 0.2 * n_pos_eff
    neg_target = int(round((pct / 100.0) * 2.0 * n_pos_eff))
    neg_target = max(1, min(neg_target, n_neg_tv))

    print(f"Target negatives for this sweep (from train/val pool): {neg_target:,}")

    neg_tv_s = neg_tv.sample(n=neg_target, random_state=RANDOM_STATE + pct)

    sweep_df = (
        pd.concat([pos_tv_s, neg_tv_s], axis=0)
          .sample(frac=1.0, random_state=RANDOM_STATE)
          .reset_index(drop=True)
    )

    print("Class counts in sweep train/val dataset:")
    print(sweep_df["burned"].value_counts())

    # Save sweep train/val parquet
    parquet_out = os.path.join(
        OUT_DIR, f"trainval_data_neg{pct:03d}pct.parquet"
    )
    sweep_df.to_parquet(parquet_out)
    print(f"Saved sweep train/val parquet for {pct}%: {parquet_out}")

    # =====================================================
    # FEATURES / TARGET FOR THIS SWEEP (TRAIN/VAL ONLY)
    # =====================================================
    X_sweep = sweep_df[predictors].copy()
    y_sweep = sweep_df["burned"].astype(np.uint8)

    # 70/20 of the *full dataset* corresponds to 70/20 out of (1 - TEST_SIZE_GLOBAL)
    val_size_inner = VAL_SIZE_OVERALL / (1.0 - TEST_SIZE_GLOBAL)  # e.g., 0.20 / 0.90
    X_train, X_val, y_train, y_val = train_test_split(
        X_sweep,
        y_sweep,
        test_size=val_size_inner,
        random_state=RANDOM_STATE,
        stratify=y_sweep,
    )

    print("\nSweep split sizes (within train/val pool):")
    print(f"  Train: {len(X_train):,}")
    print(f"  Val  : {len(X_val):,}")

    # =====================================================
    # scale_pos_weight BASED ON TRAIN SUBSET
    # =====================================================
    n_pos_train = int((y_train == 1).sum())
    n_neg_train = int((y_train == 0).sum())
    pos_weight = n_neg_train / max(1, n_pos_train)
    print(f"scale_pos_weight for this sweep (train subset): {pos_weight:.3f}")

    model = lgb.LGBMClassifier(
        **LGB_PARAMS,
        random_state=RANDOM_STATE,
        scale_pos_weight=pos_weight,
    )

    evals_result = {}
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_names=["train", "validation"],
        eval_metric=["aucpr", lgb_iou_metric_skl],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=50),
            lgb.record_evaluation(evals_result),
        ]
    )

    # =====================================================
    # LEARNING CURVE: IoU (TRAIN vs VAL)
    # =====================================================
    if "IoU" in evals_result.get("train", {}):
        train_iou_curve = evals_result["train"]["IoU"]
        val_iou_curve   = evals_result["validation"]["IoU"]

        plt.figure(figsize=(8, 5))
        plt.plot(train_iou_curve, label="Train IoU")
        plt.plot(val_iou_curve,   label="Validation IoU")
        plt.xlabel("Boosting Rounds")
        plt.ylabel("IoU (Jaccard)")
        plt.title(
            f"Option 2: Train vs Val IoU\n"
            f"NEG={pct}% (100% = 2x positives in train/val), THRESH_INIT={THRESH_INIT:.2f}"
        )
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        iou_fig_out = os.path.join(
            OUT_DIR, f"iou_curve_neg{pct:03d}pct.png"
        )
        plt.savefig(iou_fig_out, dpi=150)
        plt.close()
        print(f"Saved IoU learning curve for {pct}%: {iou_fig_out}")

    # =====================================================
    # THRESHOLD SELECTION ON VALIDATION
    # =====================================================
    y_val_proba = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
    prec, rec, thr = precision_recall_curve(y_val, y_val_proba)
    mask = rec[:-1] >= RECALL_FLOOR

    if not np.any(mask):
        print(f"\nNo threshold meets recall >= {RECALL_FLOOR:.2f}; using global max precision.")
        best_idx = np.argmax(prec[:-1])
    else:
        best_idx_rel = np.argmax(prec[:-1][mask])
        best_idx = np.flatnonzero(mask)[best_idx_rel]

    best_thr = float(thr[best_idx])  # probability threshold in [0,1]
    print(
        f"\nChosen threshold on VALID (NEG={pct}%): {best_thr:.3f}  "
        f"(precision={prec[best_idx]:.4f}, recall={rec[best_idx]:.4f})"
    )

    # =====================================================
    # FINAL METRICS ON FIXED GLOBAL TEST SET
    # =====================================================
    y_test_proba = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
    y_test_hat   = (y_test_proba >= best_thr).astype(np.uint8)

    test_iou  = jaccard_score(y_test, y_test_hat, average="binary", zero_division=0)
    test_prec = precision_score(y_test, y_test_hat, zero_division=0)
    test_rec  = recall_score(y_test, y_test_hat, zero_division=0)
    test_f1   = f1_score(y_test, y_test_hat, zero_division=0)

    print("\n==== FINAL TEST METRICS (fixed global test set) ====")
    print(f"NEG % in train/val (100%=2x pos): {pct}")
    print(f"Threshold    : {best_thr:.3f}")
    print(f"IoU (Jaccard): {test_iou:.2f}")
    print(f"Precision    : {test_prec:.2f}")
    print(f"Recall       : {test_rec:.2f}")
    print(f"F1 Score     : {test_f1:.2f}")

    summary_rows.append(
        dict(
            neg_percent=pct,
            n_pos_train=n_pos_train,
            n_neg_train=n_neg_train,
            scale_pos_weight=round(pos_weight, 3),
            threshold=round(best_thr, 3),
            test_iou=round(test_iou, 2),
            test_precision=round(test_prec, 2),
            test_recall=round(test_rec, 2),
            test_f1=round(test_f1, 2),
            best_iteration=int(model.best_iteration_ if hasattr(model, "best_iteration_") else -1),
        )
    )

    # =====================================================
    # FEATURE IMPORTANCE
    # =====================================================
    gain_imp = model.booster_.feature_importance(importance_type="gain")
    gain_imp = gain_imp / (gain_imp.sum() + 1e-12)
    feat_names = np.array(X_train.columns)

    order = np.argsort(gain_imp)[::-1][:TOP_N_IMPORT]
    plt.figure(figsize=(9, max(5, 0.28 * len(order))))
    plt.barh(feat_names[order][::-1], gain_imp[order][::-1])
    plt.xlabel("Relative Gain Importance")
    plt.title(
        f"Option 2: Feature Importance (Top {len(order)})\n"
        f"NEG={pct}% (100% = 2x positives in train/val)"
    )
    plt.tight_layout()

    fi_fig_out = os.path.join(
        OUT_DIR, f"feature_importance_neg{pct:03d}pct.png"
    )
    plt.savefig(fi_fig_out, dpi=150)
    plt.close()
    print(f"Saved feature importance plot for {pct}%: {fi_fig_out}")

# =====================================================
# SAVE SUMMARY CSV (rounded metrics)
# =====================================================
if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = os.path.join(OUT_DIR, "option2_neg_ratio_sweep_globaltest_metrics.csv")
    summary_df.to_csv(summary_csv, index=False)
    print(f"\nSaved Option 2 global-test sweep summary to: {summary_csv}")
else:
    print("\nNo sweeps were run; summary not saved.")


Loading: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction_balanced.parquet
Dropped 0 rows with NaNs/±inf; 172,072 remain.

Class counts before any splitting:
0    112224
1     59848
Name: burned, dtype: int64

Treating 'b1' as pandas 'category'.

Predictor columns: 15

Global split sizes (true distribution in test):
  Train/Val pool: 154,864
  Test (fixed)  : 17,208

Test set class counts (true distribution):
0    11223
1     5985
Name: burned, dtype: int64

Train/Val pool class counts:
0    101001
1     53863
Name: burned, dtype: int64

Effective positive samples in train/val sweeps: 53,863

== Option 2: NEGATIVE PERCENT = 10% (100% = 2x positives in TRAIN/VAL) ==
Target negatives for this sweep (from train/val pool): 10,773
Class counts in sweep train/val dataset:
1    53863
0    10773
Name: burned, dtype: int64
Saved sweep train/val parquet for 10%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option

Option 2 penalize false positives more

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Option 2 (global fixed test set) — Penalize False Positives:
- Read full cems_with_fraction_balanced.parquet
- Build features, coerce to numeric, drop bad rows
- Create a single global test set (10%), remaining 90% is train/val pool
- Fix a positive sample set once
- Sweep negative ratios (10..100%, where 100% = 2x positives)
- For each neg% sweep, ALSO sweep negative class weights to penalize false positives
- Train LightGBM (sklearn API)
- Choose threshold on validation by maximizing RECALL subject to PRECISION ≥ PRECISION_FLOOR
  (fall back to max precision if no point meets the floor)
- Evaluate on the SAME fixed global test set
- Save per (neg_percent, neg_class_weight):
    - train/val parquet
    - IoU learning curve PNG
    - feature importance PNG
    - model + metadata:
        saved_model_neg{pct:03d}pct_w{w}/
          - lgb_model_neg{pct:03d}pct_w{w}.txt
          - lgb_sklearn_neg{pct:03d}pct_w{w}.pkl
          - feature_importance_neg{pct:03d}pct_w{w}.csv
          - model_meta_neg{pct:03d}pct_w{w}.json
- Also save summary CSV across all sweeps
"""

import os
import json
import numpy as np
import pandas as pd
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve,
    jaccard_score, precision_score, recall_score, f1_score
)

# =====================================================
# CONFIG
# =====================================================
PARQUET_IN    = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction.parquet"
RANDOM_STATE  = 42

# Overall split: global test set (10%), rest train/val pool (90%)
TEST_SIZE_GLOBAL = 0.10
VAL_SIZE_OVERALL = 0.20   # overall val fraction of full dataset; derive inner val from this

THRESH_INIT   = 0.50      # used for IoU logging metric
RECALL_FLOOR  = 0.80      # legacy recall requirement (still printed)
PRECISION_FLOOR = 0.80    # precision floor to penalize false positives

TOP_N_IMPORT  = 30

# Negative ratio sweep within the train/val pool:
# 100% = 2x positives, 10% = 0.2x positives, etc.
PCT_STEPS = list(range(10, 101, 10))

# Optional: cap positives in train/val pool for speed (None = use all)
MAX_SAMPLES_POS_TRAINVAL = None

# Class-weight sweep to penalize negatives (false positives)
USE_CLASS_WEIGHT = True
NEG_CLASS_WEIGHT_SWEEP = [1.0, 2.0, 3.0, 5.0]  # try higher if precision is still low

# LightGBM base params
LGB_PARAMS = dict(
    objective="binary",
    boosting_type="gbdt",
    learning_rate=0.05,
    num_leaves=48,
    min_data_in_leaf=100,
    feature_fraction=0.75,
    bagging_fraction=0.75,
    bagging_freq=5,
    lambda_l2=2.0,
    n_jobs=-1,
)

# Output root and option-specific dir
OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest"
OUT_DIR  = os.path.join(OUT_ROOT, "option2_scale_pos_weight_penalize_fp")
os.makedirs(OUT_DIR, exist_ok=True)

print("\n=== OUTPUT DIRECTORY ===")
print(OUT_DIR)

# =====================================================
# LOAD & EARLY CLEAN (ONCE)
# =====================================================
print(f"\nLoading: {PARQUET_IN}")
df = pd.read_parquet(PARQUET_IN)

if "fraction" not in df.columns:
    raise ValueError("Expected column 'fraction' in dataset.")

df["fraction"] = df["fraction"].astype("float32").clip(0.0, 1.0)
before = len(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any").copy()
print(f"Dropped {before - len(df):,} rows with NaNs/±inf; {len(df):,} remain.")

# =====================================================
# FRACTION -> BINARY
# =====================================================
df["burned"] = (df["fraction"] > THRESH_INIT).astype(np.uint8)

print("\nClass counts before any splitting:")
print(df["burned"].value_counts(dropna=False))

# =====================================================
# FEATURE MATRIX (GLOBAL) + TYPE COERCION
# =====================================================
drop_cols = {"fraction", "burned", "bin", "year", "month", "latitude", "longitude"}
predictors = [c for c in df.columns if c not in drop_cols]

X_full = df[predictors].copy()
y_full = df["burned"].astype(np.uint8)

# Treat 'b1' as category if present
if "b1" in X_full.columns and not pd.api.types.is_categorical_dtype(X_full["b1"]):
    X_full["b1"] = X_full["b1"].astype("category")
    print("\nTreating 'b1' as pandas 'category'.")

# Coerce non-category columns to numeric and drop rows with NaNs
coerced = 0
for c in X_full.columns:
    if c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]):
        continue
    if not np.issubdtype(X_full[c].dtype, np.number):
        X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
        coerced += 1

if coerced:
    pre = len(X_full)
    num_cols = [c for c in X_full.columns if not (c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]))]
    mask = X_full[num_cols].notna().all(axis=1)
    if "b1" in X_full.columns and pd.api.types.is_categorical_dtype(X_full["b1"]):
        mask &= X_full["b1"].notna()
    X_full = X_full.loc[mask].copy()
    y_full = y_full.loc[X_full.index]
    print(f"Coerced {coerced} column(s); dropped {pre - len(X_full):,} rows post-coercion.")

print(f"\nPredictor columns: {len(X_full.columns)}")

# Combine into a single DataFrame for convenient splitting
data = X_full.copy()
data["burned"] = y_full

# Capture global categorical levels for 'b1' (for inference)
b1_categories = None
if "b1" in data.columns and pd.api.types.is_categorical_dtype(data["b1"]):
    b1_categories = list(data["b1"].cat.categories.astype(str))

# =====================================================
# GLOBAL TRAINVAL / TEST SPLIT (FIXED TEST SET)
# =====================================================
idx_trainval, idx_test = train_test_split(
    data.index,
    test_size=TEST_SIZE_GLOBAL,
    random_state=RANDOM_STATE,
    stratify=data["burned"],
)

trainval = data.loc[idx_trainval].copy()
test     = data.loc[idx_test].copy()

print("\nGlobal split sizes (true distribution in test):")
print(f"  Train/Val pool: {len(trainval):,}")
print(f"  Test (fixed)  : {len(test):,}")
print("\nTest set class counts (true distribution):")
print(test["burned"].value_counts())

# Extract global test X, y
X_test = test[predictors].copy()
y_test = test["burned"].astype(np.uint8)

# =====================================================
# TRAIN/VAL POOL: POS/NEG SPLITTING BASE
# =====================================================
pos_tv = trainval[trainval["burned"] == 1]
neg_tv = trainval[trainval["burned"] == 0]

n_pos_tv = len(pos_tv)
n_neg_tv = len(neg_tv)

print("\nTrain/Val pool class counts:")
print(trainval["burned"].value_counts())

if n_pos_tv == 0 or n_neg_tv == 0:
    raise ValueError("Train/Val pool has only one class; cannot proceed.")

# Fix positive sample within train/val pool (optionally capped)
target_pos = n_pos_tv
if MAX_SAMPLES_POS_TRAINVAL is not None:
    target_pos = min(target_pos, MAX_SAMPLES_POS_TRAINVAL)

pos_tv_s = pos_tv.sample(n=min(n_pos_tv, target_pos), random_state=RANDOM_STATE)
n_pos_eff = len(pos_tv_s)
print(f"\nEffective positive samples in train/val sweeps: {n_pos_eff:,}")

# =====================================================
# Custom IoU metric for logging (sklearn API)
# =====================================================
def lgb_iou_metric_skl(y_true, y_pred):
    y_hat = (y_pred >= THRESH_INIT).astype(np.uint8)
    iou = jaccard_score(y_true, y_hat, average="binary", zero_division=0)
    return ("IoU", iou, True)

summary_rows = []

# =====================================================
# SWEEP OVER NEGATIVE RATIOS IN TRAIN/VAL POOL
# =====================================================
for pct in PCT_STEPS:
    print("\n" + "=" * 70)
    print(f"== Option 2: NEGATIVE PERCENT = {pct}% (100% = 2x positives in TRAIN/VAL) ==")

    # 100% -> 2 * n_pos_eff
    # 10%  -> 0.2 * n_pos_eff
    neg_target = int(round((pct / 100.0) * 2.0 * n_pos_eff))
    neg_target = max(1, min(neg_target, n_neg_tv))

    print(f"Target negatives for this sweep (from train/val pool): {neg_target:,}")

    neg_tv_s = neg_tv.sample(n=neg_target, random_state=RANDOM_STATE + pct)

    sweep_df_base = (
        pd.concat([pos_tv_s, neg_tv_s], axis=0)
          .sample(frac=1.0, random_state=RANDOM_STATE)
          .reset_index(drop=True)
    )

    print("Class counts in sweep base train/val dataset:")
    print(sweep_df_base["burned"].value_counts())

    # 70/20 of the *full dataset* corresponds to 70/20 out of (1 - TEST_SIZE_GLOBAL)
    val_size_inner = VAL_SIZE_OVERALL / (1.0 - TEST_SIZE_GLOBAL)  # e.g., 0.20 / 0.90

    # =====================================================
    # NEGATIVE CLASS WEIGHT SWEEP (penalize FPs)
    # =====================================================
    for neg_w in NEG_CLASS_WEIGHT_SWEEP:
        print("\n" + "-" * 50)
        print(f"[NEG%={pct}] Training with class_weight: {{0: {neg_w}, 1: 1.0}}")

        # copy to avoid any accidental in-place mutations
        sweep_df = sweep_df_base.copy()

        # Save sweep train/val parquet for traceability
        parquet_out = os.path.join(
            OUT_DIR, f"trainval_data_neg{pct:03d}pct_w{neg_w}.parquet"
        )
        sweep_df.to_parquet(parquet_out)
        print(f"Saved sweep train/val parquet: {parquet_out}")

        # Features / target for this sweep
        X_sweep = sweep_df[predictors].copy()
        y_sweep = sweep_df["burned"].astype(np.uint8)

        X_train, X_val, y_train, y_val = train_test_split(
            X_sweep,
            y_sweep,
            test_size=val_size_inner,
            random_state=RANDOM_STATE,
            stratify=y_sweep,
        )

        if USE_CLASS_WEIGHT:
            model = lgb.LGBMClassifier(
                **LGB_PARAMS,
                random_state=RANDOM_STATE,
                class_weight={0: neg_w, 1: 1.0},  # penalize FP
            )
        else:
            # Fallback to original pos-weighting if desired
            n_pos_train = int((y_train == 1).sum())
            n_neg_train = int((y_train == 0).sum())
            pos_weight = n_neg_train / max(1, n_pos_train)
            model = lgb.LGBMClassifier(
                **LGB_PARAMS,
                random_state=RANDOM_STATE,
                scale_pos_weight=pos_weight,
            )

        evals_result = {}
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_names=["train", "validation"],
            eval_metric=["aucpr", lgb_iou_metric_skl],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=50),
                lgb.record_evaluation(evals_result),
            ]
        )

        # ---------------- LEARNING CURVE: IoU (TRAIN vs VAL) ----------------
        if "IoU" in evals_result.get("train", {}):
            train_iou_curve = evals_result["train"]["IoU"]
            val_iou_curve   = evals_result["validation"]["IoU"]

            plt.figure(figsize=(8, 5))
            plt.plot(train_iou_curve, label="Train IoU")
            plt.plot(val_iou_curve,   label="Validation IoU")
            plt.xlabel("Boosting Rounds")
            plt.ylabel("IoU (Jaccard)")
            plt.title(
                f"Train vs Val IoU\nNEG={pct}% (100% = 2x pos), "
                f"THRESH_INIT={THRESH_INIT:.2f}, w_neg={neg_w}"
            )
            plt.legend()
            plt.grid(True)
            plt.tight_layout()

            iou_fig_out = os.path.join(
                OUT_DIR, f"iou_curve_neg{pct:03d}pct_w{neg_w}.png"
            )
            plt.savefig(iou_fig_out, dpi=150)
            plt.close()
            print(f"Saved IoU learning curve: {iou_fig_out}")

        # ---------------- Threshold selection with a PRECISION FLOOR ----------------
        y_val_proba = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
        prec, rec, thr = precision_recall_curve(y_val, y_val_proba)

        # Prefer thresholds that meet the precision floor; among them choose max recall
        mask = prec[:-1] >= PRECISION_FLOOR
        if np.any(mask):
            best_idx_rel = np.argmax(rec[:-1][mask])
            best_idx = np.flatnonzero(mask)[best_idx_rel]
        else:
            # If nothing meets the floor, fall back to global max precision
            best_idx = np.argmax(prec[:-1])

        best_thr = float(thr[best_idx])
        print(
            f"Chosen threshold (PREC≥{PRECISION_FLOOR:.2f}): {best_thr:.3f} "
            f"(precision={prec[best_idx]:.4f}, recall={rec[best_idx]:.4f})  "
            f"[RECALL_FLOOR ref={RECALL_FLOOR:.2f}]"
        )

        # ---------------- FINAL METRICS ON FIXED GLOBAL TEST SET ----------------
        y_test_proba = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
        y_test_hat   = (y_test_proba >= best_thr).astype(np.uint8)

        test_iou  = jaccard_score(y_test, y_test_hat, average="binary", zero_division=0)
        test_prec = precision_score(y_test, y_test_hat, zero_division=0)
        test_rec  = recall_score(y_test, y_test_hat, zero_division=0)
        test_f1   = f1_score(y_test, y_test_hat, zero_division=0)

        best_iteration = int(model.best_iteration_ if hasattr(model, "best_iteration_") else -1)

        print(
            f"[NEG%={pct}, w_neg={neg_w}] TEST: IoU={test_iou:.3f}, "
            f"Prec={test_prec:.3f}, Rec={test_rec:.3f}, F1={test_f1:.3f}, Thr={best_thr:.3f}"
        )

        summary_rows.append(
            dict(
                neg_percent=pct,
                neg_class_weight=neg_w if USE_CLASS_WEIGHT else 1.0,
                threshold=round(best_thr, 3),
                test_iou=round(test_iou, 3),
                test_precision=round(test_prec, 3),
                test_recall=round(test_rec, 3),
                test_f1=round(test_f1, 3),
                best_iteration=best_iteration,
                precision_floor=PRECISION_FLOOR,
                recall_floor=RECALL_FLOOR,
            )
        )

        # ---------------- FEATURE IMPORTANCE ----------------
        gain_imp = model.booster_.feature_importance(importance_type="gain")
        gain_imp = gain_imp / (gain_imp.sum() + 1e-12)
        feat_names = np.array(X_train.columns)

        order = np.argsort(gain_imp)[::-1][:TOP_N_IMPORT]
        plt.figure(figsize=(9, max(5, 0.28 * len(order))))
        plt.barh(feat_names[order][::-1], gain_imp[order][::-1])
        plt.xlabel("Relative Gain Importance")
        plt.title(
            f"Feature Importance (Top {len(order)})\nNEG={pct}% (100% = 2x pos), w_neg={neg_w}"
        )
        plt.tight_layout()

        fi_fig_out = os.path.join(
            OUT_DIR, f"feature_importance_neg{pct:03d}pct_w{neg_w}.png"
        )
        plt.savefig(fi_fig_out, dpi=150)
        plt.close()
        print(f"Saved feature importance plot: {fi_fig_out}")

        # ---------------- SAVE MODEL + METADATA FOR THIS COMBO ----------------
        w_tag = int(round(float(neg_w)))
        save_dir = os.path.join(
            OUT_DIR, f"saved_model_neg{pct:03d}pct_w{w_tag}"
        )
        os.makedirs(save_dir, exist_ok=True)

        # Booster text
        booster_txt = os.path.join(
            save_dir, f"lgb_model_neg{pct:03d}pct_w{w_tag}.txt"
        )
        model.booster_.save_model(booster_txt)

        # Sklearn wrapper
        model_pkl = os.path.join(
            save_dir, f"lgb_sklearn_neg{pct:03d}pct_w{w_tag}.pkl"
        )
        joblib.dump(model, model_pkl)

        # Feature importance CSV
        fi_csv = os.path.join(
            save_dir, f"feature_importance_neg{pct:03d}pct_w{w_tag}.csv"
        )
        pd.DataFrame({
            "feature": feat_names[order],
            "gain_importance": gain_imp[order]
        }).to_csv(fi_csv, index=False)

        # Metadata JSON
        meta = {
            "option": 2,
            "neg_percent": int(pct),
            "neg_class_weight": float(neg_w),
            "threshold": round(float(best_thr), 6),
            "metrics_test": {
                "iou": round(float(test_iou), 4),
                "precision": round(float(test_prec), 4),
                "recall": round(float(test_rec), 4),
                "f1": round(float(test_f1), 4)
            },
            "best_iteration": best_iteration,
            "random_state": RANDOM_STATE,
            "precision_floor": PRECISION_FLOOR,
            "recall_floor": RECALL_FLOOR,
            "thresh_init_for_logging": THRESH_INIT,
            "predictors": predictors,
            "categorical": {
                "b1_categories": b1_categories
            },
            "paths": {
                "booster_txt": booster_txt,
                "sklearn_pkl": model_pkl,
                "feature_importance_csv": fi_csv
            }
        }

        meta_json = os.path.join(
            save_dir, f"model_meta_neg{pct:03d}pct_w{w_tag}.json"
        )
        with open(meta_json, "w") as f:
            json.dump(meta, f, indent=2)

        print(f"[Saved model artifacts for NEG={pct}%, w={w_tag}] -> {save_dir}")

# =====================================================
# SAVE SUMMARY CSV (rounded metrics)
# =====================================================
if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = os.path.join(
        OUT_DIR, f"option2_neg_ratio_sweep_globaltest_metrics_pf{str(PRECISION_FLOOR).replace('.', '')}.csv"
    )
    summary_df.to_csv(summary_csv, index=False)
    print(f"\nSaved Option 2 global-test sweep summary to:\n{summary_csv}")
else:
    print("\nNo sweeps were run; summary not saved.")



=== OUTPUT DIRECTORY ===
/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option2_scale_pos_weight_penalize_fp

Loading: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction_balanced.parquet
Dropped 0 rows with NaNs/±inf; 172,072 remain.

Class counts before any splitting:
0    112224
1     59848
Name: burned, dtype: int64

Treating 'b1' as pandas 'category'.

Predictor columns: 15

Global split sizes (true distribution in test):
  Train/Val pool: 154,864
  Test (fixed)  : 17,208

Test set class counts (true distribution):
0    11223
1     5985
Name: burned, dtype: int64

Train/Val pool class counts:
0    101001
1     53863
Name: burned, dtype: int64

Effective positive samples in train/val sweeps: 53,863

== Option 2: NEGATIVE PERCENT = 10% (100% = 2x positives in TRAIN/VAL) ==
Target negatives for this sweep (from train/val pool): 10,773
Class counts in sweep base train/val dataset:
1    53863
0    10773
Nam

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Option 2 (global fixed test set) — Penalize False Positives:
- Read full cems_with_fraction_balanced.parquet
- Build features, coerce to numeric, drop bad rows
- Create a single global test set (10%) with true 0/1 distribution (no resampling)
- Use the remaining 90% as a train/val pool
- Fix a positive sample set once
- Sweep negative ratios (10..100%, where 100% = 2x positives)
- For each neg% sweep, ALSO sweep negative class weights to penalize false positives
- Train LightGBM (sklearn API)
- Choose threshold on validation by maximizing RECALL subject to PRECISION ≥ floor
  (fall back to max precision if no point meets the floor)
- Evaluate on the SAME fixed global test set
- Save:
    - Balanced train/val parquet per (neg%, neg_class_weight) sweep
    - IoU learning curve PNG per sweep
    - Feature importance PNG per sweep
    - Per-sweep y_test_proba_*.npy for PR curves
    - Global y_test.npy once for PR curves
    - Summary CSV (rounded) across all sweeps
"""

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve,
    jaccard_score, precision_score, recall_score, f1_score
)

# =====================================================
# CONFIG
# =====================================================
PARQUET_IN    = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction.parquet"
RANDOM_STATE  = 42

# Overall split: global test set (10%), rest train/val pool (90%)
TEST_SIZE_GLOBAL = 0.10
VAL_SIZE_OVERALL = 0.20   # overall val fraction of full dataset; derive inner val from this

THRESH_INIT   = 0.50      # used for IoU logging metric
RECALL_FLOOR  = 0.80      # legacy recall requirement (still printed)
PRECISION_FLOOR = 0.80    # precision floor to penalize false positives (tighten to 0.85–0.90 if desired)

TOP_N_IMPORT  = 30

# Negative ratio sweep within the train/val pool:
# 100% = 2x positives, 10% = 0.2x positives, etc.
PCT_STEPS = list(range(10, 101, 10))

# Optional: cap positives in train/val pool for speed (None = use all)
MAX_SAMPLES_POS_TRAINVAL = None

# Class-weight sweep to penalize negatives (false positives)
USE_CLASS_WEIGHT = True
NEG_CLASS_WEIGHT_SWEEP = [1.0, 2.0, 3.0, 5.0]  # try higher if precision is still low

# LightGBM base params
LGB_PARAMS = dict(
    objective="binary",
    boosting_type="gbdt",
    learning_rate=0.05,
    num_leaves=48,
    min_data_in_leaf=100,
    feature_fraction=0.75,
    bagging_fraction=0.75,
    bagging_freq=5,
    lambda_l2=2.0,
    n_jobs=-1,
)

# Output root and option-specific dir
OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest"
OUT_DIR  = os.path.join(OUT_ROOT, "option2_scale_pos_weight_penalize_fp")
os.makedirs(OUT_DIR, exist_ok=True)

print("\n=== OUTPUT DIRECTORY ===")
print(OUT_DIR)

# =====================================================
# LOAD & EARLY CLEAN (ONCE)
# =====================================================
print(f"\nLoading: {PARQUET_IN}")
df = pd.read_parquet(PARQUET_IN)

if "fraction" not in df.columns:
    raise ValueError("Expected column 'fraction' in dataset.")

df["fraction"] = df["fraction"].astype("float32").clip(0.0, 1.0)
before = len(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any").copy()
print(f"Dropped {before - len(df):,} rows with NaNs/±inf; {len(df):,} remain.")

# =====================================================
# FRACTION -> BINARY
# =====================================================
df["burned"] = (df["fraction"] > THRESH_INIT).astype(np.uint8)

print("\nClass counts before any splitting:")
print(df["burned"].value_counts(dropna=False))

# =====================================================
# FEATURE MATRIX (GLOBAL) + TYPE COERCION
# =====================================================
drop_cols = {"fraction", "burned", "bin", "year", "month", "latitude", "longitude"}
predictors = [c for c in df.columns if c not in drop_cols]

X_full = df[predictors].copy()
y_full = df["burned"].astype(np.uint8)

# Treat 'b1' as category if present
if "b1" in X_full.columns and not pd.api.types.is_categorical_dtype(X_full["b1"]):
    X_full["b1"] = X_full["b1"].astype("category")
    print("\nTreating 'b1' as pandas 'category'.")

# Coerce non-category columns to numeric and drop rows with NaNs
coerced = 0
for c in X_full.columns:
    if c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]):
        continue
    if not np.issubdtype(X_full[c].dtype, np.number):
        X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
        coerced += 1

if coerced:
    pre = len(X_full)
    num_cols = [c for c in X_full.columns if not (c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]))]
    mask = X_full[num_cols].notna().all(axis=1)
    if "b1" in X_full.columns and pd.api.types.is_categorical_dtype(X_full["b1"]):
        mask &= X_full["b1"].notna()
    X_full = X_full.loc[mask].copy()
    y_full = y_full.loc[X_full.index]
    print(f"Coerced {coerced} column(s); dropped {pre - len(X_full):,} rows post-coercion.")

print(f"\nPredictor columns: {len(X_full.columns)}")

# Combine into a single DataFrame for convenient splitting
data = X_full.copy()
data["burned"] = y_full

# =====================================================
# GLOBAL TRAINVAL / TEST SPLIT (FIXED TEST SET)
# =====================================================
idx_trainval, idx_test = train_test_split(
    data.index,
    test_size=TEST_SIZE_GLOBAL,
    random_state=RANDOM_STATE,
    stratify=data["burned"],
)

trainval = data.loc[idx_trainval].copy()
test     = data.loc[idx_test].copy()

print("\nGlobal split sizes (true distribution in test):")
print(f"  Train/Val pool: {len(trainval):,}")
print(f"  Test (fixed)  : {len(test):,}")
print("\nTest set class counts (true distribution):")
print(test["burned"].value_counts())

# Extract global test X, y
X_test = test[predictors].copy()
y_test = test["burned"].astype(np.uint8)

# --- NEW: save global test labels once for PR curves ---
np.save(os.path.join(OUT_DIR, "y_test.npy"), y_test.values)
print(f"[SAVE] y_test -> {os.path.join(OUT_DIR, 'y_test.npy')}")

# =====================================================
# TRAIN/VAL POOL: POS/NEG SPLITTING BASE
# =====================================================
pos_tv = trainval[trainval["burned"] == 1]
neg_tv = trainval[trainval["burned"] == 0]

n_pos_tv = len(pos_tv)
n_neg_tv = len(neg_tv)

print("\nTrain/Val pool class counts:")
print(trainval["burned"].value_counts())

if n_pos_tv == 0 or n_neg_tv == 0:
    raise ValueError("Train/Val pool has only one class; cannot proceed.")

# Fix positive sample within train/val pool (optionally capped)
target_pos = n_pos_tv
if MAX_SAMPLES_POS_TRAINVAL is not None:
    target_pos = min(target_pos, MAX_SAMPLES_POS_TRAINVAL)

pos_tv_s = pos_tv.sample(n=min(n_pos_tv, target_pos), random_state=RANDOM_STATE)
n_pos_eff = len(pos_tv_s)
print(f"\nEffective positive samples in train/val sweeps: {n_pos_eff:,}")

# =====================================================
# Custom IoU metric for logging (sklearn API)
# =====================================================
def lgb_iou_metric_skl(y_true, y_pred):
    y_hat = (y_pred >= THRESH_INIT).astype(np.uint8)
    iou = jaccard_score(y_true, y_hat, average="binary", zero_division=0)
    return ("IoU", iou, True)

summary_rows = []

# =====================================================
# SWEEP OVER NEGATIVE RATIOS IN TRAIN/VAL POOL
# =====================================================
for pct in PCT_STEPS:
    print("\n" + "=" * 70)
    print(f"== Option 2: NEGATIVE PERCENT = {pct}% (100% = 2x positives in TRAIN/VAL) ==")

    # 100% -> 2 * n_pos_eff
    # 10%  -> 0.2 * n_pos_eff
    neg_target = int(round((pct / 100.0) * 2.0 * n_pos_eff))
    neg_target = max(1, min(neg_target, n_neg_tv))

    print(f"Target negatives for this sweep (from train/val pool): {neg_target:,}")

    neg_tv_s = neg_tv.sample(n=neg_target, random_state=RANDOM_STATE + pct)

    sweep_df_base = (
        pd.concat([pos_tv_s, neg_tv_s], axis=0)
          .sample(frac=1.0, random_state=RANDOM_STATE)
          .reset_index(drop=True)
    )

    print("Class counts in sweep base train/val dataset:")
    print(sweep_df_base["burned"].value_counts())

    # 70/20 of the *full dataset* corresponds to 70/20 out of (1 - TEST_SIZE_GLOBAL)
    val_size_inner = VAL_SIZE_OVERALL / (1.0 - TEST_SIZE_GLOBAL)  # e.g., 0.20 / 0.90

    # =====================================================
    # NEGATIVE CLASS WEIGHT SWEEP (penalize FPs)
    # =====================================================
    for neg_w in NEG_CLASS_WEIGHT_SWEEP:
        print("\n" + "-" * 50)
        print(f"[NEG%={pct}] Training with class_weight: {{0: {neg_w}, 1: 1.0}}")

        # copy to avoid any accidental in-place mutations
        sweep_df = sweep_df_base.copy()

        # Save sweep train/val parquet for traceability
        parquet_out = os.path.join(
            OUT_DIR, f"trainval_data_neg{pct:03d}pct_w{neg_w}.parquet"
        )
        sweep_df.to_parquet(parquet_out)
        print(f"Saved sweep train/val parquet: {parquet_out}")

        # Features / target for this sweep
        X_sweep = sweep_df[predictors].copy()
        y_sweep = sweep_df["burned"].astype(np.uint8)

        X_train, X_val, y_train, y_val = train_test_split(
            X_sweep,
            y_sweep,
            test_size=val_size_inner,
            random_state=RANDOM_STATE,
            stratify=y_sweep,
        )

        if USE_CLASS_WEIGHT:
            model = lgb.LGBMClassifier(
                **LGB_PARAMS,
                random_state=RANDOM_STATE,
                class_weight={0: neg_w, 1: 1.0},  # penalize FP
            )
        else:
            # Fallback to original pos-weighting if desired
            n_pos_train = int((y_train == 1).sum())
            n_neg_train = int((y_train == 0).sum())
            pos_weight = n_neg_train / max(1, n_pos_train)
            model = lgb.LGBMClassifier(
                **LGB_PARAMS,
                random_state=RANDOM_STATE,
                scale_pos_weight=pos_weight,
            )

        evals_result = {}
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_names=["train", "validation"],
            eval_metric=["aucpr", lgb_iou_metric_skl],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=50),
                lgb.record_evaluation(evals_result),
            ]
        )

        # ---------------- LEARNING CURVE: IoU (TRAIN vs VAL) ----------------
        if "IoU" in evals_result.get("train", {}):
            train_iou_curve = evals_result["train"]["IoU"]
            val_iou_curve   = evals_result["validation"]["IoU"]

            plt.figure(figsize=(8, 5))
            plt.plot(train_iou_curve, label="Train IoU")
            plt.plot(val_iou_curve,   label="Validation IoU")
            plt.xlabel("Boosting Rounds")
            plt.ylabel("IoU (Jaccard)")
            plt.title(
                f"Train vs Val IoU\nNEG={pct}% (100% = 2x pos), THRESH_INIT={THRESH_INIT:.2f}, w_neg={neg_w}"
            )
            plt.legend()
            plt.grid(True)
            plt.tight_layout()

            iou_fig_out = os.path.join(
                OUT_DIR, f"iou_curve_neg{pct:03d}pct_w{neg_w}.png"
            )
            plt.savefig(iou_fig_out, dpi=150)
            plt.close()
            print(f"Saved IoU learning curve: {iou_fig_out}")

        # ---------------- Threshold selection with a PRECISION FLOOR ----------------
        y_val_proba = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
        prec, rec, thr = precision_recall_curve(y_val, y_val_proba)

        # Prefer thresholds that meet the precision floor; among them choose max recall
        mask = prec[:-1] >= PRECISION_FLOOR
        if np.any(mask):
            best_idx_rel = np.argmax(rec[:-1][mask])
            best_idx = np.flatnonzero(mask)[best_idx_rel]
        else:
            # If nothing meets the floor, fall back to global max precision
            best_idx = np.argmax(prec[:-1])

        best_thr = float(thr[best_idx])
        print(f"Chosen threshold (PREC≥{PRECISION_FLOOR:.2f}): {best_thr:.3f} "
              f"(precision={prec[best_idx]:.4f}, recall={rec[best_idx]:.4f})  "
              f"[RECALL_FLOOR ref={RECALL_FLOOR:.2f}]")

        # ---------------- FINAL METRICS ON FIXED GLOBAL TEST SET ----------------
        y_test_proba = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
        y_test_hat   = (y_test_proba >= best_thr).astype(np.uint8)

        # --- NEW: save per-sweep probabilities for PR curves later ---
        proba_path = os.path.join(OUT_DIR, f"y_test_proba_neg{pct:03d}_w{neg_w}.npy")
        np.save(proba_path, y_test_proba)
        print(f"[SAVE] y_test_proba -> {proba_path}")

        test_iou  = jaccard_score(y_test, y_test_hat, average="binary", zero_division=0)
        test_prec = precision_score(y_test, y_test_hat, zero_division=0)
        test_rec  = recall_score(y_test, y_test_hat, zero_division=0)
        test_f1   = f1_score(y_test, y_test_hat, zero_division=0)

        print(f"[NEG%={pct}, w_neg={neg_w}] TEST: IoU={test_iou:.3f}, "
              f"Prec={test_prec:.3f}, Rec={test_rec:.3f}, F1={test_f1:.3f}, Thr={best_thr:.3f}")

        summary_rows.append(
            dict(
                neg_percent=pct,
                neg_class_weight=neg_w if USE_CLASS_WEIGHT else 1.0,
                threshold=round(best_thr, 3),
                test_iou=round(test_iou, 3),
                test_precision=round(test_prec, 3),
                test_recall=round(test_rec, 3),
                test_f1=round(test_f1, 3),
                best_iteration=int(model.best_iteration_ if hasattr(model, "best_iteration_") else -1),
                precision_floor=PRECISION_FLOOR,
                recall_floor=RECALL_FLOOR,
            )
        )

        # ---------------- FEATURE IMPORTANCE ----------------
        gain_imp = model.booster_.feature_importance(importance_type="gain")
        gain_imp = gain_imp / (gain_imp.sum() + 1e-12)
        feat_names = np.array(X_train.columns)

        order = np.argsort(gain_imp)[::-1][:TOP_N_IMPORT]
        plt.figure(figsize=(9, max(5, 0.28 * len(order))))
        plt.barh(feat_names[order][::-1], gain_imp[order][::-1])
        plt.xlabel("Relative Gain Importance")
        plt.title(
            f"Feature Importance (Top {len(order)})\nNEG={pct}% (100% = 2x pos), w_neg={neg_w}"
        )
        plt.tight_layout()

        fi_fig_out = os.path.join(
            OUT_DIR, f"feature_importance_neg{pct:03d}pct_w{neg_w}.png"
        )
        plt.savefig(fi_fig_out, dpi=150)
        plt.close()
        print(f"Saved feature importance plot: {fi_fig_out}")

# =====================================================
# SAVE SUMMARY CSV (rounded metrics)
# =====================================================
if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = os.path.join(
        OUT_DIR, f"option2_neg_ratio_sweep_globaltest_metrics_pf{str(PRECISION_FLOOR).replace('.', '')}.csv"
    )
    summary_df.to_csv(summary_csv, index=False)
    print(f"\nSaved Option 2 global-test sweep summary to:\n{summary_csv}")
else:
    print("\nNo sweeps were run; summary not saved.")



=== OUTPUT DIRECTORY ===
/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option2_scale_pos_weight_penalize_fp

Loading: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction_balanced.parquet
Dropped 0 rows with NaNs/±inf; 172,072 remain.

Class counts before any splitting:
0    112224
1     59848
Name: burned, dtype: int64

Treating 'b1' as pandas 'category'.

Predictor columns: 15

Global split sizes (true distribution in test):
  Train/Val pool: 154,864
  Test (fixed)  : 17,208

Test set class counts (true distribution):
0    11223
1     5985
Name: burned, dtype: int64
[SAVE] y_test -> /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option2_scale_pos_weight_penalize_fp/y_test.npy

Train/Val pool class counts:
0    101001
1     53863
Name: burned, dtype: int64

Effective positive samples in train/val sweeps: 53,863

== Option 2: NEGATIVE PERCENT = 10% (100% = 2x

Option 3. Fixed train/val ratio, sweep scale_pos_weight, same global test set

Here we:

Use the same global splits logic as above.

In the train/val pool, build a single fixed-ratio dataset (e.g., 1:1 0:1).

Sweep scale_pos_weight values on that same dataset.

Always evaluate on the same global test set.

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Option 3 (global fixed test set, fixed train/val ratio, sweep scale_pos_weight):
- Same global preprocessing & test split as Option 2
- In train/val pool:
    - Build a fixed-ratio dataset (NEG_MULT * positives)
    - Use that same dataset for all runs
- Sweep scale_pos_weight over POS_WEIGHTS
- For each weight:
    - Train model, pick threshold on val (max precision with recall ≥ floor)
    - Evaluate on the fixed global test set
- Save:
    - Fixed train/val parquet
    - IoU learning curves per weight
    - Feature importance per weight
    - Summary CSV of test metrics over weights (rounded)
"""

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve,
    jaccard_score, precision_score, recall_score, f1_score
)

# =====================================================
# CONFIG
# =====================================================
PARQUET_IN    = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction.parquet"
RANDOM_STATE  = 42

TEST_SIZE_GLOBAL = 0.10
VAL_SIZE_OVERALL = 0.20

THRESH_INIT   = 0.50          # for IoU logging during training
RECALL_FLOOR  = 0.80
TOP_N_IMPORT  = 30

# Fixed negative:positive ratio in train/val pool
NEG_MULT = 1.0   # 1.0 => 1:1, 2.0 => 2:1, etc.

# Sweep of scale_pos_weight on that fixed dataset
POS_WEIGHTS = [1.0, 2.0, 4.0, 8.0, 16.0]

# Optionally cap positives in train/val pool for speed (None = use all)
MAX_SAMPLES_POS_TRAINVAL = None

LGB_PARAMS = dict(
    objective="binary",
    boosting_type="gbdt",
    learning_rate=0.05,
    num_leaves=48,
    min_data_in_leaf=100,
    feature_fraction=0.75,
    bagging_fraction=0.75,
    bagging_freq=5,
    lambda_l2=2.0,
    n_jobs=-1,
)

OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest"
OUT_DIR  = os.path.join(OUT_ROOT, "option3_weight_sweeps")
os.makedirs(OUT_DIR, exist_ok=True)

# =====================================================
# LOAD & PREP (SAME AS OPTION 2)
# =====================================================
print(f"Loading: {PARQUET_IN}")
df = pd.read_parquet(PARQUET_IN)

if "fraction" not in df.columns:
    raise ValueError("Expected column 'fraction' in dataset.")

df["fraction"] = df["fraction"].astype("float32").clip(0.0, 1.0)
before = len(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any").copy()
print(f"Dropped {before - len(df):,} rows with NaNs/±inf; {len(df):,} remain.")

df["burned"] = (df["fraction"] > THRESH_INIT).astype(np.uint8)

print("\nClass counts before any splitting:")
print(df["burned"].value_counts(dropna=False))

drop_cols = {"fraction", "burned", "bin", "year", "month", "latitude", "longitude"}
predictors = [c for c in df.columns if c not in drop_cols]

X_full = df[predictors].copy()
y_full = df["burned"].astype(np.uint8)

if "b1" in X_full.columns and not pd.api.types.is_categorical_dtype(X_full["b1"]):
    X_full["b1"] = X_full["b1"].astype("category")
    print("\nTreating 'b1' as pandas 'category'.")

coerced = 0
for c in X_full.columns:
    if c == "b1" and pd.api.types.is_categorical_dtype(X_full[c]):
        continue
    if not np.issubdtype(X_full[c].dtype, np.number):
        X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
        coerced += 1

if coerced:
    pre = len(X_full)
    num_cols = [c for c in X_full.columns if not (c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]))]
    mask = X_full[num_cols].notna().all(axis=1)
    if "b1" in X_full.columns and pd.api.types.is_categorical_dtype(X_full["b1"]):
        mask &= X_full["b1"].notna()
    X_full = X_full.loc[mask].copy()
    y_full = y_full.loc[X_full.index]
    print(f"Coerced {coerced} column(s); dropped {pre - len(X_full):,} rows post-coercion.")

print(f"\nPredictor columns: {len(X_full.columns)}")

data = X_full.copy()
data["burned"] = y_full

# Global split
idx_trainval, idx_test = train_test_split(
    data.index,
    test_size=TEST_SIZE_GLOBAL,
    random_state=RANDOM_STATE,
    stratify=data["burned"],
)

trainval = data.loc[idx_trainval].copy()
test     = data.loc[idx_test].copy()

print("\nGlobal split sizes (true distribution in test):")
print(f"  Train/Val pool: {len(trainval):,}")
print(f"  Test (fixed)  : {len(test):,}")
print("\nTest set class counts:")
print(test["burned"].value_counts())

X_test = test[predictors].copy()
y_test = test["burned"].astype(np.uint8)

pos_tv = trainval[trainval["burned"] == 1]
neg_tv = trainval[trainval["burned"] == 0]

n_pos_tv = len(pos_tv)
n_neg_tv = len(neg_tv)
print("\nTrain/Val pool class counts:")
print(trainval["burned"].value_counts())

if n_pos_tv == 0 or n_neg_tv == 0:
    raise ValueError("Train/Val pool has only one class; cannot proceed.")

target_pos = n_pos_tv
if MAX_SAMPLES_POS_TRAINVAL is not None:
    target_pos = min(target_pos, MAX_SAMPLES_POS_TRAINVAL)

pos_tv_s = pos_tv.sample(n=min(n_pos_tv, target_pos), random_state=RANDOM_STATE)
n_pos_eff = len(pos_tv_s)
print(f"\nEffective positive samples in train/val fixed dataset: {n_pos_eff:,}")

neg_target = int(round(NEG_MULT * n_pos_eff))
neg_target = max(1, min(neg_target, n_neg_tv))
print(f"Fixed negative target in train/val (NEG_MULT={NEG_MULT}): {neg_target:,}")

neg_tv_s = neg_tv.sample(n=neg_target, random_state=RANDOM_STATE + 1)

fixed_tv_df = (
    pd.concat([pos_tv_s, neg_tv_s], axis=0)
      .sample(frac=1.0, random_state=RANDOM_STATE)
      .reset_index(drop=True)
)

print("\nClass counts in fixed train/val dataset:")
print(fixed_tv_df["burned"].value_counts())

# Save base train/val dataset
base_parquet = os.path.join(OUT_DIR, "trainval_data_fixed_ratio.parquet")
fixed_tv_df.to_parquet(base_parquet)
print(f"Saved fixed-ratio train/val parquet: {base_parquet}")

# =====================================================
# FEATURES / TARGET (train/val fixed dataset)
# =====================================================
X_tv_full = fixed_tv_df[predictors].copy()
y_tv_full = fixed_tv_df["burned"].astype(np.uint8)

print(f"\nPredictor columns (train/val fixed): {len(X_tv_full.columns)}")

# Custom IoU metric
def lgb_iou_metric_skl(y_true, y_pred):
    y_hat = (y_pred >= THRESH_INIT).astype(np.uint8)
    iou = jaccard_score(y_true, y_hat, average="binary", zero_division=0)
    return ("IoU", iou, True)

summary_rows = []

# =====================================================
# SWEEP OVER scale_pos_weight
# =====================================================
for pw in POS_WEIGHTS:
    print("\n" + "=" * 70)
    print(f"== Option 3: scale_pos_weight = {pw:.3f} on fixed train/val dataset ==")

    # inner val fraction so that overall val ≈ VAL_SIZE_OVERALL
    val_size_inner = VAL_SIZE_OVERALL / (1.0 - TEST_SIZE_GLOBAL)
    X_train, X_val, y_train, y_val = train_test_split(
        X_tv_full,
        y_tv_full,
        test_size=val_size_inner,
        random_state=RANDOM_STATE,
        stratify=y_tv_full,
    )

    print("Sweep split sizes (fixed dataset):")
    print(f"  Train: {len(X_train):,}")
    print(f"  Val  : {len(X_val):,}")

    model = lgb.LGBMClassifier(
        **LGB_PARAMS,
        random_state=RANDOM_STATE,
        scale_pos_weight=pw,
    )

    evals_result = {}
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_names=["train", "validation"],
        eval_metric=["aucpr", lgb_iou_metric_skl],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=50),
            lgb.record_evaluation(evals_result),
        ]
    )

    # IoU learning curve
    if "IoU" in evals_result.get("train", {}):
        train_iou_curve = evals_result["train"]["IoU"]
        val_iou_curve   = evals_result["validation"]["IoU"]

        plt.figure(figsize=(8, 5))
        plt.plot(train_iou_curve, label="Train IoU")
        plt.plot(val_iou_curve,   label="Validation IoU")
        plt.xlabel("Boosting Rounds")
        plt.ylabel("IoU (Jaccard)")
        plt.title(
            f"Option 3: Train vs Val IoU\n"
            f"scale_pos_weight={pw:.3f}, THRESH_INIT={THRESH_INIT:.2f}"
        )
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        safe_pw = str(pw).replace(".", "p")
        iou_fig_out = os.path.join(
            OUT_DIR, f"iou_curve_scale_pos_weight_{safe_pw}.png"
        )
        plt.savefig(iou_fig_out, dpi=150)
        plt.close()
        print(f"Saved IoU curve for pw={pw:.3f}: {iou_fig_out}")

    # Threshold on validation
    y_val_proba = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
    prec, rec, thr = precision_recall_curve(y_val, y_val_proba)
    mask = rec[:-1] >= RECALL_FLOOR

    if not np.any(mask):
        print(f"\nNo threshold meets recall >= {RECALL_FLOOR:.2f}; using global max precision.")
        best_idx = np.argmax(prec[:-1])
    else:
        best_idx_rel = np.argmax(prec[:-1][mask])
        best_idx = np.flatnonzero(mask)[best_idx_rel]

    best_thr = float(thr[best_idx])  # probability threshold in [0,1]
    print(
        f"\nChosen threshold on VALID (scale_pos_weight={pw:.3f}): {best_thr:.3f}  "
        f"(precision={prec[best_idx]:.4f}, recall={rec[best_idx]:.4f})"
    )

    # Final metrics on fixed global test
    y_test_proba = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
    y_test_hat   = (y_test_proba >= best_thr).astype(np.uint8)

    test_iou  = jaccard_score(y_test, y_test_hat, average="binary", zero_division=0)
    test_prec = precision_score(y_test, y_test_hat, zero_division=0)
    test_rec  = recall_score(y_test, y_test_hat, zero_division=0)
    test_f1   = f1_score(y_test, y_test_hat, zero_division=0)

    print("\n==== FINAL TEST METRICS (fixed global test set) ====")
    print(f"scale_pos_weight: {pw:.3f}")
    print(f"Threshold      : {best_thr:.3f}")
    print(f"IoU (Jaccard)  : {test_iou:.2f}")
    print(f"Precision      : {test_prec:.2f}")
    print(f"Recall         : {test_rec:.2f}")
    print(f"F1 Score       : {test_f1:.2f}")

    # counts from the fixed train/val dataset (same for all weights)
    n_pos_tv_fixed = int((y_tv_full == 1).sum())
    n_neg_tv_fixed = int((y_tv_full == 0).sum())

    summary_rows.append(
        dict(
            scale_pos_weight=pw,
            n_pos_trainval=n_pos_tv_fixed,
            n_neg_trainval=n_neg_tv_fixed,
            threshold=round(best_thr, 3),
            test_iou=round(test_iou, 2),
            test_precision=round(test_prec, 2),
            test_recall=round(test_rec, 2),
            test_f1=round(test_f1, 2),
            best_iteration=int(model.best_iteration_ if hasattr(model, "best_iteration_") else -1),
        )
    )

    # Feature importance
    gain_imp = model.booster_.feature_importance(importance_type="gain")
    gain_imp = gain_imp / (gain_imp.sum() + 1e-12)
    feat_names = np.array(X_train.columns)

    order = np.argsort(gain_imp)[::-1][:TOP_N_IMPORT]
    plt.figure(figsize=(9, max(5, 0.28 * len(order))))
    plt.barh(feat_names[order][::-1], gain_imp[order][::-1])
    plt.xlabel("Relative Gain Importance")
    plt.title(
        f"Option 3: Feature Importance (Top {len(order)})\n"
        f"scale_pos_weight={pw:.3f}"
    )
    plt.tight_layout()

    fi_fig_out = os.path.join(
        OUT_DIR, f"feature_importance_scale_pos_weight_{safe_pw}.png"
    )
    plt.savefig(fi_fig_out, dpi=150)
    plt.close()
    print(f"Saved feature importance plot for pw={pw:.3f}: {fi_fig_out}")

# Summary CSV (rounded metrics)
if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = os.path.join(OUT_DIR, "option3_weight_sweeps_globaltest_metrics.csv")
    summary_df.to_csv(summary_csv, index=False)
    print(f"\nSaved Option 3 global-test sweep summary to: {summary_csv}")
else:
    print("\nNo sweeps were run; summary not saved.")


Loading: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction_balanced.parquet
Dropped 0 rows with NaNs/±inf; 172,072 remain.

Class counts before any splitting:
0    112224
1     59848
Name: burned, dtype: int64

Treating 'b1' as pandas 'category'.

Predictor columns: 15

Global split sizes (true distribution in test):
  Train/Val pool: 154,864
  Test (fixed)  : 17,208

Test set class counts:
0    11223
1     5985
Name: burned, dtype: int64

Train/Val pool class counts:
0    101001
1     53863
Name: burned, dtype: int64

Effective positive samples in train/val fixed dataset: 53,863
Fixed negative target in train/val (NEG_MULT=1.0): 53,863

Class counts in fixed train/val dataset:
0    53863
1    53863
Name: burned, dtype: int64
Saved fixed-ratio train/val parquet: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option3_weight_sweeps/trainval_data_fixed_ratio.parquet

Predictor columns (train/val fixed): 15

=

Option 4 – Negative-ratio sweeps + focal loss, same global test set

This mirrors Option 2’s negative-ratio sweeps in the train/val pool, but uses a focal loss custom objective via LightGBM’s train API. It shares the same global test split logic.



In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Option 4 (Weighted log loss, global fixed test set) with robust fallback:
- Try LightGBM core API with custom weighted log loss via fobj.
- If LightGBM lacks `fobj`, fall back to XGBoost with the same weighted loss.
- Threshold stored as a true probability in [0,1]; metrics rounded in CSV.

Outputs under:
.../neg_ratio_experiments_globaltest/option4_weighted_logloss/
"""

import os, inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve,
    jaccard_score,
    precision_score,
    recall_score,
    f1_score,
)

# ----------------- CONFIG -----------------
PARQUET_IN    = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction.parquet"
RANDOM_STATE  = 42

TEST_SIZE_GLOBAL = 0.10
VAL_SIZE_OVERALL = 0.20

THRESH_INIT   = 0.50     # only for IoU logging / feval
RECALL_FLOOR  = 0.80
TOP_N_IMPORT  = 30

# 10%..400% (100% ≈ 2x negatives per positive; 400% ≈ 8x)
PCT_STEPS     = list(range(10, 401, 10))
MAX_SAMPLES_POS_TRAINVAL = None

# Weight for false negatives in custom loss
FN_WEIGHT = 10.0

LGB_PARAMS = dict(
    boosting_type="gbdt",
    learning_rate=0.05,
    num_leaves=48,
    min_data_in_leaf=100,
    feature_fraction=0.75,
    bagging_fraction=0.75,
    bagging_freq=5,
    lambda_l2=2.0,
    n_jobs=-1,
    metric="aucpr",
)

OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest"
OUT_DIR  = os.path.join(OUT_ROOT, "option4_weighted_logloss")
os.makedirs(OUT_DIR, exist_ok=True)

# ----------------- Helpers -----------------
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def lgb_has_fobj():
    try:
        import inspect as _inspect
        sig = _inspect.signature(lgb.train)
        return "fobj" in sig.parameters
    except Exception:
        return False

# --- WEIGHTED LOG LOSS for LightGBM (margin -> grad/hess) ---
def weighted_logloss_lgb(y_pred, dataset):
    """
    Custom weighted log loss for LightGBM:
    - Higher weight on positives (to penalize false negatives).
    """
    y_true = dataset.get_label()
    pred_prob = sigmoid(y_pred)

    grad = pred_prob - y_true
    grad[y_true == 1] *= FN_WEIGHT

    hess = pred_prob * (1.0 - pred_prob)
    hess[y_true == 1] *= FN_WEIGHT

    return grad, hess

# Same math for XGBoost (preds, dtrain signature)
def weighted_logloss_xgb(preds, dtrain):
    y_true = dtrain.get_label()
    pred_prob = sigmoid(preds)

    grad = pred_prob - y_true
    grad[y_true == 1] *= FN_WEIGHT

    hess = pred_prob * (1.0 - pred_prob)
    hess[y_true == 1] *= FN_WEIGHT

    return grad, hess

def iou_metric_lgb(y_pred, dataset):
    y_true = dataset.get_label()
    y_hat  = (sigmoid(y_pred) >= THRESH_INIT).astype(np.uint8)
    iou    = jaccard_score(y_true, y_hat, average="binary", zero_division=0)
    return "IoU", float(iou), True

# ----------------- LOAD & PREP -----------------
print(f"Loading: {PARQUET_IN}")
df = pd.read_parquet(PARQUET_IN)
if "fraction" not in df.columns:
    raise ValueError("Expected column 'fraction' in dataset.")

df["fraction"] = df["fraction"].astype("float32").clip(0, 1)
before = len(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any").copy()
print(f"Dropped {before - len(df):,} rows with NaNs/±inf; {len(df):,} remain.")

df["burned"] = (df["fraction"] > THRESH_INIT).astype(np.uint8)
print("\nClass counts before any splitting:")
print(df["burned"].value_counts(dropna=False))

drop_cols = {"fraction", "burned", "bin", "year", "month", "latitude", "longitude"}
predictors = [c for c in df.columns if c not in drop_cols]

X_full = df[predictors].copy()
y_full = df["burned"].astype(np.uint8)

if "b1" in X_full.columns and not pd.api.types.is_categorical_dtype(X_full["b1"]):
    X_full["b1"] = X_full["b1"].astype("category")
    print("\nTreating 'b1' as pandas 'category'.")

coerced = 0
for c in X_full.columns:
    if c == "b1" and pd.api.types.is_categorical_dtype(X_full[c]):
        continue
    if not np.issubdtype(X_full[c].dtype, np.number):
        X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
        coerced += 1

if coerced:
    pre = len(X_full)
    num_cols = [c for c in X_full.columns if not (c == "b1" and pd.api.types.is_categorical_dtype(X_full["b1"]))]
    mask = X_full[num_cols].notna().all(axis=1)
    if "b1" in X_full.columns and pd.api.types.is_categorical_dtype(X_full["b1"]):
        mask &= X_full["b1"].notna()
    X_full = X_full.loc[mask].copy()
    y_full = y_full.loc[X_full.index]
    print(f"Coerced {coerced} column(s); dropped {pre - len(X_full):,} rows post-coercion.")
print(f"\nPredictor columns: {len(X_full.columns)}")

data = X_full.copy()
data["burned"] = y_full

# Global split (fixed test with true distribution)
idx_trainval, idx_test = train_test_split(
    data.index,
    test_size=TEST_SIZE_GLOBAL,
    random_state=RANDOM_STATE,
    stratify=data["burned"],
)
trainval = data.loc[idx_trainval].copy()
test     = data.loc[idx_test].copy()

print("\nGlobal split sizes (true distribution in test):")
print(f"  Train/Val pool: {len(trainval):,}")
print(f"  Test (fixed)  : {len(test):,}")
print("\nTest set class counts:")
print(test["burned"].value_counts())

X_test = test[predictors].copy()
y_test = test["burned"].astype(np.uint8)

# Test set class counts (constant across sweeps)
test_pos = int((y_test == 1).sum())
test_neg = int((y_test == 0).sum())
print(f"\nTest set positives (1): {test_pos:,}")
print(f"Test set negatives (0): {test_neg:,}")

pos_tv = trainval[trainval["burned"] == 1]
neg_tv = trainval[trainval["burned"] == 0]
n_pos_tv, n_neg_tv = len(pos_tv), len(neg_tv)
print("\nTrain/Val pool class counts:")
print(trainval["burned"].value_counts())
if n_pos_tv == 0 or n_neg_tv == 0:
    raise ValueError("Train/Val pool has only one class; cannot proceed.")

# Fix positive subset in train/val pool (optionally capped)
target_pos = n_pos_tv if MAX_SAMPLES_POS_TRAINVAL is None else min(n_pos_tv, MAX_SAMPLES_POS_TRAINVAL)
pos_tv_s = pos_tv.sample(n=target_pos, random_state=RANDOM_STATE)
n_pos_eff = len(pos_tv_s)
print(f"\nEffective positive samples in train/val sweeps: {n_pos_eff:,}")

USE_LGB_FOBJ = lgb_has_fobj()
if not USE_LGB_FOBJ:
    print("\n[INFO] LightGBM build lacks `fobj` support on train(); falling back to XGBoost for weighted log loss.")
    import xgboost as xgb  # imported only when needed

summary_rows = []

# ----------------- SWEEP -----------------
for pct in PCT_STEPS:
    print("\n" + "=" * 70)
    print(f"== Option 4 (Weighted log loss): NEGATIVE PERCENT = {pct}% (100% = 2x positives in train/val) ==")

    neg_target = int(round((pct / 100.0) * 2.0 * n_pos_eff))
    neg_target = max(1, min(neg_target, n_neg_tv))
    print(f"Target negatives for this sweep: {neg_target:,}")

    neg_tv_s = neg_tv.sample(n=neg_target, random_state=RANDOM_STATE + pct)

    sweep_df = (
        pd.concat([pos_tv_s, neg_tv_s], axis=0)
        .sample(frac=1.0, random_state=RANDOM_STATE)
        .reset_index(drop=True)
    )
    print("Class counts in sweep train/val dataset:")
    print(sweep_df["burned"].value_counts())

    parquet_out = os.path.join(OUT_DIR, f"trainval_data_neg{pct:03d}pct_weightedlogloss.parquet")
    sweep_df.to_parquet(parquet_out)
    print(f"Saved sweep train/val parquet for {pct}%: {parquet_out}")

    X_sweep = sweep_df[predictors].copy()
    y_sweep = sweep_df["burned"].astype(np.uint8)

    val_size_inner = VAL_SIZE_OVERALL / (1.0 - TEST_SIZE_GLOBAL)
    X_train, X_val, y_train, y_val = train_test_split(
        X_sweep, y_sweep,
        test_size=val_size_inner,
        random_state=RANDOM_STATE,
        stratify=y_sweep,
    )

    print("\nSweep split sizes:")
    print(f"  Train: {len(X_train):,}")
    print(f"  Val  : {len(X_val):,}")

    evals_result = {}
    if USE_LGB_FOBJ:
        # ---------- LightGBM path with custom weighted log loss ----------
        train_set = lgb.Dataset(X_train, label=y_train)
        val_set   = lgb.Dataset(X_val, label=y_val, reference=train_set)
        params = LGB_PARAMS.copy()
        params["seed"] = RANDOM_STATE
        params["objective"] = "binary"  # overridden by fobj

        booster = lgb.train(
            params,
            train_set,
            num_boost_round=10000,
            valid_sets=[train_set, val_set],
            valid_names=["train", "validation"],
            fobj=weighted_logloss_lgb,
            feval=iou_metric_lgb,
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=50),
                lgb.record_evaluation(evals_result),
            ]
        )

        # Learning curve (IoU)
        if "IoU" in evals_result.get("train", {}):
            plt.figure(figsize=(8,5))
            plt.plot(evals_result["train"]["IoU"], label="Train IoU (weighted log loss)")
            plt.plot(evals_result["validation"]["IoU"], label="Validation IoU (weighted log loss)")
            plt.xlabel("Boosting Rounds"); plt.ylabel("IoU (Jaccard)")
            plt.title(
                f"Option 4 (Weighted log loss - LGB): Train vs Val IoU\n"
                f"NEG={pct}%  THRESH_INIT={THRESH_INIT:.2f}"
            )
            plt.legend(); plt.grid(True); plt.tight_layout()
            iou_fig_out = os.path.join(OUT_DIR, f"iou_curve_neg{pct:03d}pct_weightedlogloss.png")
            plt.savefig(iou_fig_out, dpi=150); plt.close()
            print(f"Saved IoU curve for {pct}%: {iou_fig_out}")

        # Validation probabilities (convert margins to probs)
        y_val_proba = sigmoid(booster.predict(X_val, num_iteration=booster.best_iteration))
        # ---------- end LightGBM path ----------

    else:
        # ---------- XGBoost fallback with custom objective ----------
        import xgboost as xgb

        def iou_metric_xgb(preds, dtrain):
            y = dtrain.get_label()
            y_hat = (sigmoid(preds) >= THRESH_INIT).astype(np.uint8)
            iou = jaccard_score(y, y_hat, average="binary", zero_division=0)
            return "IoU", float(iou)

        # >>>>>>>>>>>>> enable_categorical=True <<<<<<<<<<<<<<
        dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
        dval   = xgb.DMatrix(X_val,   label=y_val,   enable_categorical=True)

        params_xgb = dict(
            booster="gbtree",
            eta=0.05,
            max_depth=0,        # use `max_leaves` with tree_method=hist
            max_leaves=48,
            subsample=0.75,
            colsample_bytree=0.75,
            reg_lambda=2.0,
            tree_method="hist",   # or "gpu_hist" if you want GPU
            objective="reg:logistic",  # overridden by custom obj
            eval_metric="aucpr",
            seed=RANDOM_STATE,
            nthread=-1,
        )

        watchlist = [(dtrain, "train"), (dval, "validation")]
        booster = xgb.train(
            params_xgb,
            dtrain,
            num_boost_round=10000,
            evals=watchlist,
            obj=weighted_logloss_xgb,
            feval=iou_metric_xgb,
            verbose_eval=50,
            early_stopping_rounds=50,
        )

        # Validation probabilities (margins -> probs)
        y_val_proba = sigmoid(
            booster.predict(dval, iteration_range=(0, booster.best_iteration + 1))
        )
        # ---------- end XGBoost path ----------

    # Threshold selection (validation) with recall floor
    prec, rec, thr = precision_recall_curve(y_val, y_val_proba)
    mask = rec[:-1] >= RECALL_FLOOR
    if not np.any(mask):
        best_idx = np.argmax(prec[:-1])
    else:
        best_idx = np.flatnonzero(mask)[np.argmax(prec[:-1][mask])]
    best_thr = float(thr[best_idx])

    # Test metrics (fixed global test)
    if USE_LGB_FOBJ:
        y_test_proba = sigmoid(
            booster.predict(X_test, num_iteration=booster.best_iteration)
        )
    else:
        import xgboost as xgb
        dtest = xgb.DMatrix(X_test, enable_categorical=True)
        y_test_proba = sigmoid(
            booster.predict(dtest, iteration_range=(0, booster.best_iteration + 1))
        )

    y_test_hat = (y_test_proba >= best_thr).astype(np.uint8)
    test_iou  = jaccard_score(y_test, y_test_hat, average="binary", zero_division=0)
    test_prec = precision_score(y_test, y_test_hat, zero_division=0)
    test_rec  = recall_score(y_test, y_test_hat, zero_division=0)
    test_f1   = f1_score(y_test, y_test_hat, zero_division=0)

    print("\n==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====")
    print(f"NEG % in train/val (100%=2x pos): {pct}")
    print(f"Threshold    : {best_thr:.3f}")
    print(f"IoU (Jaccard): {test_iou:.2f}")
    print(f"Precision    : {test_prec:.2f}")
    print(f"Recall       : {test_rec:.2f}")
    print(f"F1 Score     : {test_f1:.2f}")

    summary_rows.append(
        dict(
            neg_percent=pct,
            n_pos_train=int((y_sweep == 1).sum()),
            n_neg_train=int((y_sweep == 0).sum()),
            test_pos=test_pos,          # number of 1s in test set
            test_neg=test_neg,          # number of 0s in test set
            fn_weight=FN_WEIGHT,
            threshold=round(best_thr, 3),        # 0–1
            test_iou=round(test_iou, 2),
            test_precision=round(test_prec, 2),
            test_recall=round(test_rec, 2),
            test_f1=round(test_f1, 2),
            best_iteration=int(
                getattr(booster, "best_iteration", getattr(booster, "best_ntree_limit", 0))
            ),
            backend=("lightgbm" if USE_LGB_FOBJ else "xgboost"),
        )
    )

    # Feature importance (gain)
    if USE_LGB_FOBJ:
        gain_imp = booster.feature_importance(importance_type="gain")
        feat_names = np.array(X_train.columns)
    else:
        fmap = booster.get_score(importance_type="gain")
        feat_names = np.array(X_train.columns)
        gain_imp = np.array(
            [fmap.get(f"f{i}", 0.0) for i in range(len(feat_names))],
            dtype=float,
        )

    gain_imp = gain_imp / (gain_imp.sum() + 1e-12)
    order = np.argsort(gain_imp)[::-1][:TOP_N_IMPORT]

    plt.figure(figsize=(9, max(5, 0.28 * len(order))))
    plt.barh(feat_names[order][::-1], gain_imp[order][::-1])
    plt.xlabel("Relative Gain Importance")
    plt.title(
        f"Option 4 (Weighted log loss, {('LGB' if USE_LGB_FOBJ else 'XGB')}): "
        f"Feature Importance (Top {len(order)})\nNEG={pct}% (100% = 2x positives)"
    )
    plt.tight_layout()
    fi_fig_out = os.path.join(OUT_DIR, f"feature_importance_neg{pct:03d}pct_weightedlogloss.png")
    plt.savefig(fi_fig_out, dpi=150)
    plt.close()
    print(f"Saved weighted-logloss feature importance plot for {pct}%: {fi_fig_out}")

# ----------------- SUMMARY CSV -----------------
if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv = os.path.join(OUT_DIR, "option4_weightedlogloss_neg_ratio_sweep_globaltest_metrics.csv")
    summary_df.to_csv(summary_csv, index=False)
    print(f"\nSaved Option 4 (weighted log loss) global-test sweep summary to: {summary_csv}")
else:
    print("\nNo sweeps were run; summary not saved.")


Loading: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/cems_with_fraction_balanced.parquet
Dropped 0 rows with NaNs/±inf; 172,072 remain.

Class counts before any splitting:
0    112224
1     59848
Name: burned, dtype: int64

Treating 'b1' as pandas 'category'.

Predictor columns: 15

Global split sizes (true distribution in test):
  Train/Val pool: 154,864
  Test (fixed)  : 17,208

Test set class counts:
0    11223
1     5985
Name: burned, dtype: int64

Test set positives (1): 5,985
Test set negatives (0): 11,223

Train/Val pool class counts:
0    101001
1     53863
Name: burned, dtype: int64

Effective positive samples in train/val sweeps: 53,863

[INFO] LightGBM build lacks `fobj` support on train(); falling back to XGBoost for weighted log loss.

== Option 4 (Weighted log loss): NEGATIVE PERCENT = 10% (100% = 2x positives in train/val) ==
Target negatives for this sweep: 10,773
Class counts in sweep train/val dataset:
1    53863
0    10773
Name: burned, dtype: int6

/home/spotter5/.conda/envs/xgboost_gpu/lib/python3.9/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-aucpr:0.91805	train-IoU:0.87133	validation-aucpr:0.91032	validation-IoU:0.86775
[49]	train-aucpr:0.94049	train-IoU:0.87184	validation-aucpr:0.93016	validation-IoU:0.86892

==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====
NEG % in train/val (100%=2x pos): 10
Threshold    : 0.628
IoU (Jaccard): 0.45
Precision    : 0.47
Recall       : 0.92
F1 Score     : 0.62
Saved weighted-logloss feature importance plot for 10%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/feature_importance_neg010pct_weightedlogloss.png

== Option 4 (Weighted log loss): NEGATIVE PERCENT = 20% (100% = 2x positives in train/val) ==
Target negatives for this sweep: 21,545
Class counts in sweep train/val dataset:
1    53863
0    21545
Name: burned, dtype: int64
Saved sweep train/val parquet for 20%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_loglo

/home/spotter5/.conda/envs/xgboost_gpu/lib/python3.9/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-aucpr:0.84879	train-IoU:0.77326	validation-aucpr:0.84571	validation-IoU:0.76851
[50]	train-aucpr:0.88947	train-IoU:0.77560	validation-aucpr:0.88136	validation-IoU:0.77188

==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====
NEG % in train/val (100%=2x pos): 20
Threshold    : 0.628
IoU (Jaccard): 0.45
Precision    : 0.49
Recall       : 0.84
F1 Score     : 0.62
Saved weighted-logloss feature importance plot for 20%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/feature_importance_neg020pct_weightedlogloss.png

== Option 4 (Weighted log loss): NEGATIVE PERCENT = 30% (100% = 2x positives in train/val) ==
Target negatives for this sweep: 32,318
Class counts in sweep train/val dataset:
1    53863
0    32318
Name: burned, dtype: int64
Saved sweep train/val parquet for 30%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_loglo

...

[0]	train-aucpr:0.59049	train-IoU:0.43901	validation-aucpr:0.58961	validation-IoU:0.43798
[50]	train-aucpr:0.64526	train-IoU:0.44098	validation-aucpr:0.64010	validation-IoU:0.43948
[51]	train-aucpr:0.64587	train-IoU:0.44113	validation-aucpr:0.64059	validation-IoU:0.43959

==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====
NEG % in train/val (100%=2x pos): 380
Threshold    : 0.634
IoU (Jaccard): 0.46
Precision    : 0.48
Recall       : 0.90
F1 Score     : 0.63
Saved weighted-logloss feature importance plot for 380%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/feature_importance_neg380pct_weightedlogloss.png

== Option 4 (Weighted log loss): NEGATIVE PERCENT = 390% (100% = 2x positives in train/val) ==
Target negatives for this sweep: 101,001
Class counts in sweep train/val dataset:
0    101001
1     53863
Name: burned, dtype: int64
Saved sweep train/val parquet for 390%: /explore/nobackup/pe

/home/spotter5/.conda/envs/xgboost_gpu/lib/python3.9/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-aucpr:0.58846	train-IoU:0.42588	validation-aucpr:0.58807	validation-IoU:0.42607
[50]	train-aucpr:0.64681	train-IoU:0.44085	validation-aucpr:0.64135	validation-IoU:0.43991

==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====
NEG % in train/val (100%=2x pos): 390
Threshold    : 0.627
IoU (Jaccard): 0.46
Precision    : 0.48
Recall       : 0.90
F1 Score     : 0.63
Saved weighted-logloss feature importance plot for 390%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/feature_importance_neg390pct_weightedlogloss.png

== Option 4 (Weighted log loss): NEGATIVE PERCENT = 400% (100% = 2x positives in train/val) ==
Target negatives for this sweep: 101,001
Class counts in sweep train/val dataset:
0    101001
1     53863
Name: burned, dtype: int64
Saved sweep train/val parquet for 400%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighte

/home/spotter5/.conda/envs/xgboost_gpu/lib/python3.9/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-aucpr:0.58582	train-IoU:0.43895	validation-aucpr:0.58266	validation-IoU:0.43980
[50]	train-aucpr:0.64534	train-IoU:0.44116	validation-aucpr:0.63924	validation-IoU:0.44019
[52]	train-aucpr:0.64729	train-IoU:0.44144	validation-aucpr:0.64114	validation-IoU:0.44032

==== FINAL TEST METRICS (weighted log loss, fixed global test set) ====
NEG % in train/val (100%=2x pos): 400
Threshold    : 0.634
IoU (Jaccard): 0.46
Precision    : 0.48
Recall       : 0.90
F1 Score     : 0.63
Saved weighted-logloss feature importance plot for 400%: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/feature_importance_neg400pct_weightedlogloss.png

Saved Option 4 (weighted log loss) global-test sweep summary to: /explore/nobackup/people/spotter5/clelland_fire_ml/ml_training/neg_ratio_experiments_globaltest/option4_weighted_logloss/option4_weightedlogloss_neg_ratio_sweep_globaltest_metrics.csv
